# Miller Ensemble Machine Learning

**Author:** Dan Miller

**Date:** November 16th, 2025

**Objective:** Successfully create an ensemble model to accurately predict the quality of red wine.

## Introduction:

This project explores the Wine Quality UCI dataset, specifically the set on red wine.  With the goal of being able to create an ensemble model to predict wine quality.  The data will first be explored and prepped, then two models will be created.  The performance of the two models will be compared and a conclusion will be given at the end.

## Imports

In [1]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    VotingClassifier,
)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
